# [Enums and Pattern Matching](https://doc.rust-lang.org/book/ch06-00-enums.html)
Time to explore the world of `enum` in Rust!

## [Defining an Enum](https://doc.rust-lang.org/book/ch06-01-defining-an-enum.html#defining-an-enum)
Nothing too surprising about the syntax:
```rust
enum IpAddrKind {
    V4,
    V6,
}
```
Though the introduction is quite interesting, implying they may have more responsibilities than in other languages.

They seem to be talking about use as an interface type?

```rust
enum IpAddrKind {
    V4,
    V6,
}
...
    let four = IpAddrKind::V4;
    let six = IpAddrKind::V6;
```

So this is working similar to a namespace?

That raises a lot of questions.

```rust
fn route(ip_kind: IpAddrKind) {}
```

Well that confirms their use in inheritance.

The syntax of Rust is quite strongly tied to the base types.

Another use here is interesting:
```rust
    enum IpAddrKind {
        V4,
        V6,
    }

    struct IpAddr {
        kind: IpAddrKind,
        address: String,
    }

    let home = IpAddr {
        kind: IpAddrKind::V4,
        address: String::from("127.0.0.1"),
    };

    let loopback = IpAddr {
        kind: IpAddrKind::V6,
        address: String::from("::1"),
    };
```
It's... constructing properties in a struct?

I presume this implies the required compiler validation.

And you can just create a tuple struct within it:

```rust
    enum IpAddr {
        V4(u8, u8, u8, u8),
        V6(String),
    }
```

Wow.

And then you can easily create internal structures within the elements:
```rust
enum Message {
    Quit,
    Move { x: i32, y: i32 },
    Write(String),
    ChangeColor(i32, i32, i32),
}
```

I've gotta hand it to them, this is a nice type system.

Then you can define methods by extending the base enum:
```rust
    impl Message {
        fn call(&self) {
            // method body would be defined here
        }
    }

    let m = Message::Write(String::from("hello"));
    m.call();
```

So I guess you'd use another `impl` block for each of the class options mentioned?

What if you don't? Do they get the "default" implementation from the base class?

How do you create an abstract method that child classes are required to implement?









## [The Option Enum and Its Advantages Over Null Values](https://doc.rust-lang.org/book/ch06-01-defining-an-enum.html#the-option-enum-and-its-advantages-over-null-values)
The tutorial talks a little bit about `null` and the problems it causes. 

Then they go on to say Rust **doesn't have a null type!**

Given my strong dislike of the use of null in so many languages I'm very happy to hear that.

I've seen this type abused so much:
 * For a search with no results... just use an empty list.
 * For error conditions... use an exception.
 * For an invalid result
 * As a default input argument for optional values.
    * This might be the one I agree with, if an abstract type isn't possible.

I've used programming languages with non-nullable types or having a specific null-type.

But I don't think I've ever seen null removed entirely.

Well... it sounds good but what to they do instead for blank or missing values?

So they say there's an option enum present in the default scope:
```rust
enum Option<T> {
    None,
    Some(T),
}
    let some_number = Some(5);
    let some_char = Some('e');

    let absent_number: Option<i32> = None;
```

It allows a value or an absent one... 

So this is a specific nullable type not like NoneType.

Hmm...  

```rust
    let x: i8 = 5;
    let y: Option<i8> = Some(5);

    let sum = x + y;
```
This code fails to compile because is `Option<i8>` is a now a separate (nullable) type.

So it follows that no regular variable can contain null. 

`Option<T>` has a large number of methods built in for handling a possible null value:
 * Allowing a default value to be specified.
 * Using a method to retrieve an alternative value.
 * Raising a panic if null (discouraged).
 * Querying to check if null.
 
To sum it up:
 * All types are non-nullable by default.
 * The `Option` type allowing nulling.
     * It create a separate type to prevent mixing with non-nullable types.
     * If forces you to create handlers specifically to deal with nullable items.
     * It defines a lot of different handling methods when a null does occur.
 * I'm optimisic about this system.
 

Enough notetaking, I need to write some code to test this out. 









